<a href="https://colab.research.google.com/github/Shruti022/Healthcare-Chatbot/blob/main/LLM_project_phase_1_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Phase 1: Stepwise API Exploration

Step 1: Import Libraries


In [1]:
!pip install requests pandas
!pip install faiss-cpu sentence-transformers numpy pandas

import requests
import pandas as pd
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 108.5 MB/s eta 0:00:00


In [2]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%writefile build_embeddings.py
import pandas as pd
import numpy as np
import faiss
import json
from sentence_transformers import SentenceTransformer

# === REAL PATH (from readlink) ===
BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# ---------------------------------------------
# Load Data
# ---------------------------------------------
df = pd.read_csv(f"{BASE}/clinical_trials_diabetes_full.csv")

df["status"] = df["status"].astype(str).str.strip().str.title()
bad_status = ["Terminated", "Withdrawn", "Suspended", "No Longer Available", "Unknown"]
df_clean = df[~df["status"].isin(bad_status)].copy()

# ---------------------------------------------
# Chunking
# ---------------------------------------------
chunks = []
chunk_map = []

for idx, row in df_clean.iterrows():
    title = str(row.get("brief_title", ""))
    summary = str(row.get("brief_summary", ""))

    if len(summary) < 20:
        continue

    text = f"Title: {title}\nSummary: {summary}"
    chunks.append(text)

    chunk_map.append({
        "nct_id": row["nct_id"],
        "title": title,
        "text": text,
        "status": row["status"]
    })

print(f"Created {len(chunks)} chunks.")

# ---------------------------------------------
# Embeddings
# ---------------------------------------------
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, batch_size=64, show_progress_bar=True)

np.save(f"{BASE}/embeddings.npy", embeddings)
print("Saved embeddings.npy")

# ---------------------------------------------
# Save chunk map
# ---------------------------------------------
with open(f"{BASE}/chunk_map.json", "w") as f:
    json.dump(chunk_map, f)

print("Saved chunk_map.json")

# ---------------------------------------------
# Build & Save FAISS
# ---------------------------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))
faiss.write_index(index, f"{BASE}/faiss.index")

print("Saved faiss.index")


Writing build_embeddings.py


In [5]:
!python build_embeddings.py

2025-11-23 01:36:36.486069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763861796.505132    1490 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763861796.511008    1490 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763861796.525647    1490 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763861796.525672    1490 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763861796.525676    1490 computation_placer.cc:177] computation placer alr

In [6]:
%%writefile run_bot.py
import json, re, os
import numpy as np
import faiss
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# === REAL PATH ===
BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# === Load GEMINI API KEY ===
genai.configure(api_key=os.environ.get("GENAI_KEY"))
gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

# === Load Artifacts ===
embeddings = np.load(f"{BASE}/embeddings.npy")
index = faiss.read_index(f"{BASE}/faiss.index")

with open(f"{BASE}/chunk_map.json") as f:
    chunk_map = json.load(f)

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# ===============================================
# Agents (same as before – no changes needed)
# ===============================================

class SymptomParser:
    def __init__(self, model):
        self.model = model

    def parse(self, text):
        prompt = f"""
Extract symptoms and return ONLY JSON:
{{"symptoms":["list"],"duration":"text","context":"text"}}
Input: "{text}"
"""
        try:
            out = self.model.generate_content(prompt)
            raw = out.text.strip()
            match = re.search(r"\{.*\}", raw, re.DOTALL)
            if match:
                return json.loads(match.group(0))
        except:
            pass
        return {"symptoms":[text],"duration":"unknown","context":""}


class RetrievalAgent:
    def __init__(self, model, index, cmap):
        self.model = model
        self.index = index
        self.cmap = cmap

    def retrieve(self, parsed, top_k=5):
        symptoms = parsed.get("symptoms", [])
        query = " ".join(symptoms)

        q_emb = self.model.encode([query])
        D, I = self.index.search(q_emb.astype("float32"), top_k)

        trials = []
        seen = set()

        for idx in I[0]:
            item = self.cmap[idx]
            if item["nct_id"] not in seen:
                trials.append(item)
                seen.add(item["nct_id"])

        return {"query": query, "trials": trials}


class DiagnosisAdvisor:
    def __init__(self, model):
        self.model = model

    def advise(self, parsed, retrieved):
        evidence = "\n".join([f"{t['nct_id']}: {t['text']}" for t in retrieved["trials"]])

        prompt = f"""
Answer ONLY using clinical-trial evidence.
If evidence does not answer the question, say:
"EVIDENCE IS INSUFFICIENT."

USER:
{parsed}

EVIDENCE:
{evidence}
"""

        try:
            out = self.model.generate_content(prompt)
            return out.text.strip()
        except:
            return "Unable to generate advice."


class ActiveSafetyFilter:
    def __init__(self, model):
        self.model = model
        self.safety_cfg = {
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
        }

    def verify(self, advice, trials):
        evidence_text = "\n".join([t["text"] for t in trials])

        audit = f"""
Check the following advice for medical safety:

ADVICE:
{advice}

EVIDENCE:
{evidence_text[:4000]}

RULES:
- If it suggests stopping medication → UNSAFE
- If it diagnoses a condition → UNSAFE
- If it makes unsupported claims → UNSAFE

Respond:
SAFE
or
CORRECTED: <safer version>
"""

        try:
            out = self.model.generate_content(audit, safety_settings=self.safety_cfg)
            txt = out.text.strip()
            if txt.startswith("SAFE"):
                return advice, "Pass"
            else:
                return f"⚠️ SAFETY REVISION:\n{txt}", "Revised"
        except:
            return "⚠️ Safety filter triggered.", "Revised(API)"


# ===============================================
# HEALTHCARE BOT PIPELINE
# ===============================================
class HealthcareBot:
    def __init__(self):
        self.parser = SymptomParser(gemini_model)
        self.retriever = RetrievalAgent(embed_model, index, chunk_map)
        self.advisor = DiagnosisAdvisor(gemini_model)
        self.safety = ActiveSafetyFilter(gemini_model)

    def process(self, user_input):
        p = self.parser.parse(user_input)
        r = self.retriever.retrieve(p)
        draft = self.advisor.advise(p, r)
        final, status = self.safety.verify(draft, r["trials"])
        return {
            "recommendation": final,
            "cited_trials": [t["nct_id"] for t in r["trials"]],
            "safety_status": status
        }

bot = HealthcareBot()

def run_bot(user_input):
    return bot.process(user_input)


Writing run_bot.py


UI frontend application simple web interface

https://docs.streamlit.io/develop/tutorials/chat-and-llm-apps/build-conversational-apps

In [7]:
%%writefile app.py
import streamlit as st
from run_bot import run_bot

st.title("Clinical Trial Health Advisor 🤖")

if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Get input
if user_input := st.chat_input("Describe your symptoms..."):
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Display user msg
    with st.chat_message("user"):
        st.markdown(user_input)

    # Run pipeline
    result = run_bot(user_input)
    reply = result["recommendation"]

    # Display bot reply
    with st.chat_message("assistant"):
        st.markdown(reply)

    st.session_state.messages.append({"role": "assistant", "content": reply})


Writing app.py


In [8]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!chmod +x cloudflared

In [9]:
#AI LLM from HW4 Q3
!streamlit run app.py &>/dev/null&
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-23T01:37:32Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-23T01:37:32Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-23T01:37:35Z INF +--------------------------------------------------------------------------------------------+
2025-11-23T01:37:35Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-23T01:37:35Z INF |  https://bones-recipients-jessica-replaced.trycloudfla